In [86]:
import numpy as np
from scipy import special as sp
import csv
import copy
from constitutive import *

#need matlab to check
import matlab
import matlab.engine


In [2]:
eng = matlab.engine.start_matlab()
eng.addpath ('../domeconduit/Code/TimeDependent/', nargout= 0 )
eng.addpath ('../MassMatrix/', nargout= 0 )

In [87]:
# Model parameters
m = {}


# temperature
Tvec = np.array((950, 900, 850, 800, 750))
m['Ti'] = 3
m['T'] = np.float(Tvec[m['Ti']-1] + 273.15)  # temperature [K]

# geometry, all in meters
m['R'] = 50
m['conduitlength'] = 4e3


# densities
m['rho_s']  = 2600.
m['rho_l']  = 2200.
m['rho_hd'] = 2200.
m['rho_cd'] = 2200.

# gas constants
m['Rw'] = 461.5 # J/(kg K)
m['Rc'] = 188.9 # J/(kg K)
m['B']  = 18.02/44.01;  # molar mass ratio water/co2

# permeability 
m['kc'] = 1e-13       # magma permeability constant, m2
klw = {}
klw['top'] = 1e-14    # wall rock permeability at top, m2
klw['mi']  = 3.2      # wall rock permeability exponent in Manning & Ingebritsen 1999
klw['L']   = m['R']   # length scale for wall rock gas percolation
m['klw'] = klw


# initialise solid fraction curves
fname = 'chiscurves/chis_ti' + str(m['Ti']) + '.csv'
with open(fname, newline='') as f:
    reader = np.array(list(csv.reader(f)),dtype=np.float)

pp = {}
pp['breaks'] = np.array(reader[:,0]).reshape(len(reader),1)
pp['coefs'] = reader[:-1,1:]
m['pf'] = pp


### the next few cells are just to check that the python code for constitutive relations is the same as the matlab functions

In [112]:
# check phase fraction derivatives
p = mat_to_py(np.array([100e6,75e6,50e6,25e6]))
phi_g = mat_to_py(np.array([0.1,0.2,0.3,0.3]))
mh = mat_to_py(np.array([0.9,0.92,0.95,0.98]))

chi_s, dp, cf0, cf1, cf2 = solid_mass_frac(1e-6*p, m['pf'])
rho, c1, c2, phi_s, phi_l, rho_g = density(p, phi_g, mh, m)
h2o_d, co2_d, dh2o_d, dco2_d = solubility(p, mh, m)

dh2o_d, dco2_d = solubility_deriv(p, mh, dh2o_d, dco2_d, m)
dchisdp = solid_mass_frac_deriv(dp, cf0, cf1, cf2)
dc1, dc2, dc2dh2od, dc2dco2d = c1c2_deriv(h2o_d, co2_d, c1, c2, dh2o_d, dco2_d, m)
dphis, dphil = phasefracs_deriv(phi_s, phi_g, chi_s, c1, c2, dchisdp, dc1, dc2, m)

pyout = {}
pyout['h2o_d'] = h2o_d
pyout['co2_d'] = co2_d
pyout['c1']    = c1
pyout['c2']    = c2
pyout['chi_s'] = chi_s
pyout['phi_s'] = phi_s
pyout['phi_l'] = phi_l
pyout['rho']   = rho
pyout['dh2o_d']= dh2o_d
pyout['dco2_d']= dco2_d
pyout['dc1']   = dc1
pyout['dc2']   = dc2
pyout['dphis'] = dphis
pyout['dphil'] = dphil



In [113]:

pm = matlab.double(p.tolist())
phigm = matlab.double(phi_g.tolist())
mhm = matlab.double(mh.tolist())

mtmp = copy.deepcopy(m)

mtmp['pf']['breaks'] = matlab.double(mtmp['pf']['breaks'].tolist())
mtmp['pf']['coefs'] = matlab.double(mtmp['pf']['coefs'].tolist())

Chi_hd, Chi_cd, c1, c2, chi_s, phi_s, phi_l, rho, dChihd, dChicd, dc1, dc2, dphis, dphil = eng.phasefracderivs(pm, phigm, mhm, mtmp, nargout=14)

mlout = {}
mlout['h2o_d'] = np.array(Chi_hd)
mlout['co2_d'] = np.array(Chi_cd)
mlout['c1']    = np.array(c1)
mlout['c2']    = np.array(c2)
mlout['chi_s'] = np.array(chi_s)
mlout['phi_s'] = np.array(phi_s)
mlout['phi_l'] = np.array(phi_l)
mlout['rho']   = np.array(rho)
mlout['dh2o_d']= np.array(dChihd)
mlout['dco2_d']= np.array(dChicd)
mlout['dc1']   = np.array(dc1)
mlout['dc2']   = np.array(dc2)
mlout['dphis'] = np.array(dphis)
mlout['dphil'] = np.array(dphil)


In [119]:

print('python code: rho = ', pyout['rho'].T)
print('matlab code: rho = ', mlout['rho'].T, '\n')

print('python code: c1 = ', pyout['c1'].T)
print('matlab code: c1 = ', mlout['c1'].T, '\n')

print('python code: c2 = ', pyout['c2'].T)
print('matlab code: c2 = ', mlout['c2'].T, '\n')

print('python code: dh2o_d/dp = ', pyout['dh2o_d']['p'].T)
print('matlab code: dh2o_d/dp = ', mlout['dh2o_d'][:,0].T, '\n')

print('python code: dh2o_d/dmh = ', pyout['dh2o_d']['mh'].T)
print('matlab code: dh2o_d/dmh = ', mlout['dh2o_d'][:,1].T, '\n')

print('python code: dco2_d/dp = ', pyout['dco2_d']['p'].T)
print('matlab code: dco2_d/dp = ', mlout['dco2_d'][:,0].T, '\n')

print('python code: dco2_d/dmh = ', pyout['dco2_d']['mh'].T)
print('matlab code: dco2_d/dmh = ', mlout['dco2_d'][:,1].T, '\n')


print('python code: dc1/dp = ', pyout['dc1']['p'].T)
print('matlab code: dc1/dp = ', mlout['dc1'][:,0].T, '\n')

print('python code: dc1/dmh = ', pyout['dc1']['mh'].T)
print('matlab code: dc1/dmh = ', mlout['dc1'][:,1].T, '\n')


print('python code: dc2/dp = ', pyout['dc2']['p'].T)
print('matlab code: dc2/dp = ', mlout['dc2'][:,0].T, '\n')

print('python code: dc2/dmh = ', pyout['dc2']['mh'].T)
print('matlab code: dc2/dmh = ', mlout['dc2'][:,1].T, '\n')



print('python code: dphis/dp = ', pyout['dphis']['p'].T)
print('matlab code: dphis/dp = ', mlout['dphis'][:,0].T, '\n')

print('python code: dphis/dphig = ', pyout['dphis']['phi_g'].T)
print('matlab code: dphis/dphig = ', mlout['dphis'][:,1].T, '\n')

print('python code: dphis/dmh = ', pyout['dphis']['mh'].T)
print('matlab code: dphis/dmh = ', mlout['dphis'][:,2].T, '\n')


print('python code: dphil/dp = ', pyout['dphil']['p'].T)
print('matlab code: dphil/dp = ', mlout['dphil'][:,0].T, '\n')

print('python code: dphil/dphig = ', pyout['dphil']['phi_g'].T)
print('matlab code: dphil/dphig = ', mlout['dphil'][:,1].T, '\n')

print('python code: dphil/dmh = ', pyout['dphil']['mh'].T)
print('matlab code: dphil/dmh = ', mlout['dphil'][:,2].T, '\n')

python code: rho =  [[2159.0937166  1944.16366888 1722.2065867  1834.24726395]]
matlab code: rho =  [[2159.0937166  1944.16366888 1722.2065867  1834.24726395]] 

python code: c1 =  [[1.03796513 1.03247169 1.02613116 1.01792041]]
matlab code: c1 =  [[1.03796513 1.03247169 1.02613116 1.01792041]] 

python code: c2 =  [[0.9634235  0.96854956 0.97453429 0.98239507]]
matlab code: c2 =  [[0.9634235  0.96854956 0.97453429 0.98239507]] 

python code: dh2o_d/dp =  [[2.08740937e-10 2.38465594e-10 2.87041498e-10 3.88404630e-10]]
matlab code: dh2o_d/dp =  [2.08740937e-10 2.38465594e-10 2.87041498e-10 3.88404630e-10] 

python code: dh2o_d/dmh =  [[0.02569807 0.02094031 0.01584116 0.01013028]]
matlab code: dh2o_d/dmh =  [0.02569807 0.02094031 0.01584116 0.01013028] 

python code: dco2_d/dp =  [[6.31131807e-13 4.99306140e-13 3.12690527e-13 1.25823972e-13]]
matlab code: dco2_d/dp =  [6.31131807e-13 4.99306140e-13 3.12690527e-13 1.25823972e-13] 

python code: dco2_d/dmh =  [[-0.00062137 -0.00046553 -0.

In [120]:
# check viscosity

h2o_d = 5.
phi_s_eta = 0.5
gdot = 1.0e-5

eta1, eta_m1, eta_phi1 = viscosity(h2o_d, phi_s_eta, gdot, m)
eta2, eta_m2, eta_phi2 = eng.tdcFV('calc_eta', h2o_d*0.01, phi_s_eta, m['T'], 2, gdot, nargout=3)

print('python code: ', eta1, eta_m1, eta_phi1)
print('matlab code: ', eta2, eta_m2, eta_phi2)



python code:  233815.67501426488 5832.851929387548 40.08599529781234
matlab code:  233815.67501426488 5832.851929387548 40.08599529781234


In [121]:
# check solid mass fraction

p = mat_to_py(np.array([100e6,50e6]))

chi_s, dp, cf0, cf1, cf2 = solid_mass_frac(1e-6*p, m['pf'])
print('python code: chi_s = ', chi_s.T)
print('matlab code: chi_s = ', 0.477586596754590,0.581054531506694)


dchisdp = solid_mass_frac_deriv(dp, cf0, cf1, cf2)
print('python code: dchis/dp = 1e-8 x', 1e8*dchisdp.T)
print('matlab code: dchis/dp = 1e-8 x', -0.148494868567476,  -0.468613491456686)


python code: chi_s =  [[0.4775866  0.58105453]]
matlab code: chi_s =  0.47758659675459 0.581054531506694
python code: dchis/dp = 1e-8 x [[-0.14849487 -0.46861349]]
matlab code: dchis/dp = 1e-8 x -0.148494868567476 -0.468613491456686


In [122]:
# check permeability
degas = 1
phigperm = mat_to_py(np.array([0.316898461281632,0.330159292330150]))
z = 1e2*mat_to_py(np.array([9.400000000000000,8.066666666666666]))

klat = lateralperm(phigperm, degas, z, m)
print('python code: 1e-14 x', klat.T*1e14)
print('matlab code: 1e-14 x', 0.504717768022393, 0.609486347121373)

python code: 1e-14 x [[0.50471777 0.60948635]]
matlab code: 1e-14 x 0.504717768022393 0.609486347121373


In [123]:
eng.quit()